In [20]:
import requests, json
import pandas as pd
from urllib.parse import quote

In [21]:
#API 키 가져오기
with open('C:/workspace/kakao.txt') as file:
    road_key = file.read()

In [22]:
# URL 만들기
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = ' 제주특별자치도 서귀포시 일주동로 9209'
url = f'{base_url}?query={quote(addr)}'

In [23]:
#Header : Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {road_key}'}

In [24]:
#Kakao Local API 호출하여 결과 가져오기
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '제주특별자치도 서귀포시 법환동 841-2',
    'b_code': '5013010200',
    'h_code': '5013059000',
    'main_address_no': '841',
    'mountain_yn': 'N',
    'region_1depth_name': '제주특별자치도',
    'region_2depth_name': '서귀포시',
    'region_3depth_h_name': '대륜동',
    'region_3depth_name': '법환동',
    'sub_address_no': '2',
    'x': '126.509117350341',
    'y': '33.2483380261564'},
   'address_name': '제주특별자치도 서귀포시 일주동로 9209',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '제주특별자치도 서귀포시 일주동로 9209',
    'building_name': '이마트서귀포점',
    'main_building_no': '9209',
    'region_1depth_name': '제주특별자치도',
    'region_2depth_name': '서귀포시',
    'region_3depth_name': '법환동',
    'road_name': '일주동로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.509117350341',
    'y': '33.2483380261564',
    'zone_no': '63572'},
   'x': '126.509117350341',
   'y': '33.2483380261564'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [25]:
lat = result['documents'][0]['y'] #위도(latitude)
lng = result['documents'][0]['x'] #경도(latitude)

In [26]:
dong=result['documents'][0]['address']['region_3depth_name']
print(dong)

법환동


In [27]:
lat, lng, dong

('33.2483380261564', '126.509117350341', '법환동')

In [28]:
#제주도 마트 정보 완성하기
df = pd.read_excel('./(최종)제주도 마트 목록.xlsx')
df

,마트명,주소,위도,경도
0,이마트 서귀포점,제주특별자치도 서귀포시 일주동로 9209,33.248338,126.509117
1,이마트 신제주점,제주특별자치도 제주시 1100로 3348,33.484965,126.480432
2,롯데마트 제주점,제주특별자치도 제주시 연북로 1,33.482442,126.481834
3,이마트 제주점,제주특별자치도 제주시 탑동로 38,33.517975,126.521177
4,홈플러스 서귀포점,제주특별자치도 서귀포시 중앙로 180,33.260486,126.559935
...,...,...,...,...
369,덕성마트,제주특별자치도 서귀포시 마소물로8번길 8,33.243942,126.599786
370,달빛마트,제주특별자치도 제주시 한경면 신정로 216-1,33.318046,126.199678
371,정희마트,제주특별자치도 제주시 동문로21길 21,33.515825,126.536836
372,유신종합마트,제주특별자치도 제주시 동문로2길 9,33.512258,126.526820


In [45]:
lat_list, lng_list, dong_list= [], [], [] #빈 리스트 생성. 위도와 경도 값 저장
for i in df.index: #df의 각 행에 대해 반복
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json() #API에 요청을 보내고 응답을 JSON 형식으로 받음
    try:
        lat_list.append(result['documents'][0]['y']) #위도(y) 추출하여 lat_list에 추가
        lng_list.append(result['documents'][0]['x']) #경도(x) 추출하여 lng_list에 추가
        dong_list.append(result['documents'][0]['address']['region_3depth_name'])
    except:
        print(df.마트명[i])  
#try-except 구문: try블록 실행 중 예외 발생 시 except블록 실행   

In [32]:
#isin(): 주어진 리스트나 배열에 속하는 값들을 필터링
df[df.마트명.isin(['뉴월드마트 화북점'])].주소

97    제주특별자치도 제주시 일주동로 233 1층
Name: 주소, dtype: object

In [33]:
#카카오맵에서 직접 검색해서 주소 수정
df.iloc[97]['주소'] = '제주특별자치도 제주시 일주동로 233 1층'
df.iloc[97]

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_2536\1280500124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[97]['주소'] = '제주특별자치도 제주시 일주동로 233 1층'


마트명                  뉴월드마트 화북점
주소     제주특별자치도 제주시 일주동로 233 1층
위도            33.5197112375369
경도            126.570276881705
Name: 97, dtype: object

In [34]:
df[df.마트명.isin(['조끄뜨레마트 연동점'])].주소

133    제주특별자치도 제주시 국기로 33 1층
Name: 주소, dtype: object

In [35]:
#카카오맵에서 직접 검색해서 주소 수정
df.iloc[133]['주소'] = '제주특별자치도 제주시 국기로 33 1층'
df.iloc[133]

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_2536\35805693.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[133]['주소'] = '제주특별자치도 제주시 국기로 33 1층'


마트명               조끄뜨레마트 연동점
주소     제주특별자치도 제주시 국기로 33 1층
위도          33.4789509185905
경도          126.490108566658
Name: 133, dtype: object

In [48]:
#수정된 데이터로 다시 위도경도 받아오기
lat_list, lng_list, dong_list = [], [],[] #빈 리스트 생성. 위도와 경도 값 저장
for i in df.index: #df의 각 행에 대해 반복
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json() #API에 요청을 보내고 응답을 JSON 형식으로 받음
    try:
        lat_list.append(result['documents'][0]['y']) #위도(y) 추출하여 lat_list에 추가
        lng_list.append(result['documents'][0]['x']) #경도(x) 추출하여 lng_list에 추가
        dong_list.append(result['documents'][0]['address']['region_3depth_name'])
    except:
        print(df.마트명[i])  
#try-except 구문: try블록 실행 중 예외 발생 시 except블록 실행

In [49]:
df['위도']=lat_list
df['경도']=lng_list
df['주소2']=dong_list

In [50]:
#제주도 마트 위도 경도 동 리스트
lat_list
lng_list
dong_list

['법환동',
 '노형동',
 '노형동',
 '삼도이동',
 '동홍동',
 '도남동',
 '상효동',
 '화북일동',
 '구좌읍 송당리',
 '이도이동',
 '애월읍 하귀1리',
 '애월읍 애월리',
 '일도이동',
 '중문동',
 '대정읍 상모리',
 '한림읍 한림리',
 '성산읍 고성리',
 '조천읍 함덕리',
 '구좌읍 평대리',
 '표선면 표선리',
 '한경면 신창리',
 '남원읍 남원리',
 '토평동',
 '노형동',
 '조천읍 조천리',
 '구좌읍 김녕리',
 '조천읍 함덕리',
 '하효동',
 '애월읍 봉성리',
 '남원읍 위미리',
 '한경면 고산리',
 '아라이동',
 '동홍동',
 '서귀동',
 '서호동',
 '안덕면 사계리',
 '표선면 성읍리',
 '토평동',
 '노형동',
 '대정읍 무릉리',
 '건입동',
 '노형동',
 '도련일동',
 '삼도일동',
 '동홍동',
 '연동',
 '봉개동',
 '표선면 세화리',
 '한경면 저지리',
 '우도면 연평리',
 '이도이동',
 '연동',
 '대정읍 하모리',
 '조천읍 신촌리',
 '오라삼동',
 '이도이동',
 '표선면 가시리',
 '토평동',
 '서귀동',
 '조천읍 함덕리',
 '애월읍 신엄리',
 '대정읍 보성리',
 '연동',
 '토평동',
 '성산읍 고성리',
 '일도이동',
 '노형동',
 '남원읍 하례리',
 '도두일동',
 '외도일동',
 '노형동',
 '애월읍 곽지리',
 '도남동',
 '남원읍 신례리',
 '아라일동',
 '아라일동',
 '한림읍 한수리',
 '외도일동',
 '오라일동',
 '남원읍 하례리',
 '동홍동',
 '삼양이동',
 '조천읍 함덕리',
 '중문동',
 '애월읍 고내리',
 '조천읍 함덕리',
 '구좌읍 김녕리',
 '한림읍 동명리',
 '도련일동',
 '노형동',
 '일도이동',
 '노형동',
 '성산읍 고성리',
 '삼도일동',
 '강정동',
 '동홍동',
 '애월읍 광령리',
 '화북일동',
 '아라이동',
 '이도이동',
 '대정읍 하모리'

In [51]:
df.to_excel("(최종)제주도 마트 목록.xlsx", index=False)

In [52]:
pd.read_excel('./(최종)제주도 마트 목록.xlsx')

,마트명,주소,위도,경도,주소2
0,이마트 서귀포점,제주특별자치도 서귀포시 일주동로 9209,33.248338,126.509117,법환동
1,이마트 신제주점,제주특별자치도 제주시 1100로 3348,33.484965,126.480432,노형동
2,롯데마트 제주점,제주특별자치도 제주시 연북로 1,33.482442,126.481834,노형동
3,이마트 제주점,제주특별자치도 제주시 탑동로 38,33.517975,126.521177,삼도이동
4,홈플러스 서귀포점,제주특별자치도 서귀포시 중앙로 180,33.260486,126.559935,동홍동
...,...,...,...,...,...
369,덕성마트,제주특별자치도 서귀포시 마소물로8번길 8,33.243942,126.599786,보목동
370,달빛마트,제주특별자치도 제주시 한경면 신정로 216-1,33.318046,126.199678,한경면 고산리
371,정희마트,제주특별자치도 제주시 동문로21길 21,33.515825,126.536836,건입동
372,유신종합마트,제주특별자치도 제주시 동문로2길 9,33.512258,126.526820,일도일동
